# Задание 4. Сравнение методов классификации

1. Задание выполнить в среде Google Colaboratory, сдать в Teams данный файл.
2. Использовать выбранную и закрепленную за вами выборку данных (датасет).
3. Разбить выборку на обучающую и контрольную в пропорциях 80%-20% или 70%-30%.
4. Сравнить лучшие методы из заданий 1-3 между собой и подробно обосновать результат.
5. Дополнительным критерием оценки качества задания является его визуализация.

## Выполнил студент

ФИО

Группа

## Выборка данных <...>

* Ссылка на выборку: ...
* Описание выборки: ...
* Количество классов: ...
* Количество признаков: ...
* Типы признаков: ...
* Количество объектов: ...

## Лучшие методы классификации из предыдущих заданий

* Метрический: ...
* Логический: ...
* Линейный: ...

## Лига чемпионов

In [ ]:
...

---


# Заключение

Победителем Лиги чемпионов для выборки "..." является ...